In [28]:
import pandas as pd
import numpy as np

from funcs import clean_column_names

import random

In [2]:
gatec = pd.read_excel('../sample_data/gatec.xlsx')

In [4]:
gatec = gatec.convert_dtypes()
gatec.columns = clean_column_names(gatec.columns)

In [7]:
columns_to_select = [
    'cod_fazenda',
    'area_(ha)',
    'producao_(t)',
    'dist_(km)',
    'atr'
]

In [8]:
select_gatec = gatec[columns_to_select]

In [18]:
df_fazendas = select_gatec.groupby('cod_fazenda', as_index=False).agg({'area_(ha)': 'sum', 'producao_(t)': 'sum', 'dist_(km)': 'first', 'atr': 'first'})

In [20]:
df_fazendas['produtividade_(ton/ha)'] = df_fazendas.apply(lambda x: x['producao_(t)'] / x['area_(ha)'], axis=1)

In [32]:
df_fazendas['latitude'] = np.random.randint(14, 22, size=len(df_fazendas)) + np.random.rand(len(df_fazendas))
df_fazendas['longitude'] = np.random.randint(42, 48, size=len(df_fazendas)) + np.random.rand(len(df_fazendas))

In [59]:
df_fazendas['localizacao'] = df_fazendas.apply(lambda x: np.array([x['latitude'], x['longitude']]), axis=1)

In [56]:
df_fazendas.head()

,cod_fazenda,area_(ha),producao_(t),dist_(km),atr,produtividade_(ton/ha),latitude,longitude,localizacao
0,7,10.42,907.582,8.0,150.773056,87.100000,15.018938,45.673853,"[15.018938424921503, 45.673852671284564]"
1,8,10.05,777.87,6.0,151.808452,77.400000,14.453772,46.229189,"[14.453771855668172, 46.22918930343569]"
2,9,79.14,4165.8,18.0,150.07958,52.638362,15.616189,45.539827,"[15.616188637891852, 45.53982704865857]"
3,10,39.87,2699.199,6.0,146.631468,67.700000,21.444898,42.959761,"[21.444897520966723, 42.959760836157756]"
4,11,48.46,4677.955,3.0,121.2522,96.532295,18.540176,44.386422,"[18.54017554368559, 44.386421733228325]"


In [61]:
np.linalg.norm(df_fazendas.loc[0, 'localizacao'] - df_fazendas.loc[3, 'localizacao'])

6.975617878980207